In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install -q realesrgan  

In [ ]:
import os
import torch
from PIL import Image
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
import numpy as np

class Generator(torch.nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.model = torch.nn.Sequential(
            torch.nn.Conv2d(3, 64, kernel_size=4, stride=2, padding=1),
            torch.nn.ReLU(),
            torch.nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1),
            torch.nn.ReLU(),
            torch.nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1),
            torch.nn.ReLU(),
            torch.nn.ConvTranspose2d(64, 3, kernel_size=4, stride=2, padding=1),
            torch.nn.Tanh()
        )

    def forward(self, x):
        return self.model(x)

generator = Generator()
generator.load_state_dict(torch.load("generator.pth", map_location="cpu"))
generator.eval()

img_path = "/kaggle/input/clothes-tryon/clothes_tryon_dataset/test/image/00002_00.jpg"  # change if needed
orig_image = Image.open(img_path).convert("RGB").resize((192, 256))
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])
input_tensor = transform(orig_image).unsqueeze(0)  
with torch.no_grad():
    gan_output_tensor = generator(input_tensor).squeeze(0)
gan_output_image = ((gan_output_tensor * 0.5) + 0.5).clamp(0, 1)
gan_output_pil = transforms.ToPILImage()(gan_output_image)




In [ ]:

from realesrgan import RealESRGAN
from torchvision.transforms.functional import to_tensor

sr_model = RealESRGAN(device="cpu", scale=2)
sr_model.load_weights("RealESRGAN_x2.pth", download=True) 

sr_image = sr_model.predict(gan_output_pil)
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
plt.imshow(orig_image)
plt.title("Original")
plt.axis("off")

plt.subplot(1, 3, 2)
plt.imshow(gan_output_pil)
plt.title("GAN Output")
plt.axis("off")

plt.subplot(1, 3, 3)
plt.imshow(sr_image)
plt.title("Super-Resolved")
plt.axis("off")

plt.tight_layout()
plt.show()